In [ ]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
with open('data/config.txt', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
MAPBOX_TOKEN = data["token"]
MAPBOX_STYLE = "mapbox://styles/mapbox/dark-v10"

In [ ]:
df = pd.read_csv("data/fire_nrt_M6_100853.csv")

In [ ]:
df.head()

In [ ]:
def to_hour_str(number):
    string = '{:04d}'.format(number)
    string = string[:2] + ':' + string[2:]
    return string

In [ ]:
times = df.groupby(['acq_date'])['acq_date'].count().index.tolist()
frames_data = [df.loc[df['acq_date'] == t] for t in times]

In [ ]:
frames= [go.Frame(data=[go.Densitymapbox(lat=f['latitude'], lon=f['longitude'], z=f['brightness'], radius=10)], name=str(f.iloc[0]['acq_date'])) for f in frames_data]

In [ ]:
buttons=[
         dict(label="Play",method="animate",args=[None, {'fromcurrent':True, "transition": {"duration": 30, "easing": "quadratic-in-out"}}]),
         dict(label="Pause",method="animate",args=[[None], {"frame": {"duration": 0, "redraw": False},"mode": "immediate", "transition": {"duration": 0}}])
]


sliders_dict = {
    'active':0,
    'currentvalue': dict(font=dict(size=16), prefix='Time: ', visible=True),
    "transition": {"duration": 3000, "easing": "cubic-in-out"},
    'x': 0,
    'steps': []
}

for i,t in enumerate(times):
    slider_step = {"args": [
                        [t],
                        {"frame": {"duration": 3000, "redraw": False},
                         #"mode": "immediate",
                         "transition": {"duration": 60, "easing": "quadratic-in-out"}}
                    ],
            "label": t,
            "method": "animate",
            "value": t
    }
    sliders_dict['steps'].append(slider_step)

In [ ]:
fig = go.Figure(data = [go.Densitymapbox(lat=df['latitude'], lon=df['longitude'], z=df['brightness'], radius=1, colorscale='Hot', zmax=400, zmin=0)],
               layout=go.Layout(updatemenus=[dict(type="buttons", buttons=buttons,showactive=True)] ), 
               frames=frames
)

fig.update_layout(mapbox_style=MAPBOX_STYLE, 
                  mapbox_accesstoken=MAPBOX_TOKEN)


fig.update_layout(sliders=[sliders_dict],
                 title='Forest fires over time')

fig.update_layout(height=850)
fig.show()